In [ ]:
# Imports
import os
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.python.keras import utils
from keras.utils import to_categorical
from numba import cuda
cuda.select_device(0)
cuda.close()

In [ ]:
# Reading in the data
mel_specs = pd.read_csv('../data/genre_mel_specs_clean.csv')

In [ ]:
def get_genre_subset(data, genre_subset):

    
    df = data.loc[data['labels'].isin(genre_subset)]
    df = df.reset_index().drop(columns=['index'])
    
    # Creating a new label dictionary
    new_label_dict = {}
    for i in range(len(genre_subset)):
        new_label_dict[genre_subset[i]] = i
    
    # Changing labels to be the new labels
    df['y'] = df['labels'].map(new_label_dict)

    return df

In [ ]:
def preprocess_mel_spec_data(data, genre_subset):

    # Getting a subset of the genres using our genre_subset function
    subset = get_genre_subset(data, genre_subset)
    
    # Dropping label columns to prepare our feature vector
    specs = subset.drop(columns=['labels', 'y'])
    print(len(specs))
    # Reshaping the arrays to their original "image" form
    X = []
    for i in range(len(genre_subset)*100):
        X.append(np.array(specs.iloc[i]).reshape(128,660))
        
    # Converting list X to an array
    X = np.array(X)
    
    # Defining our targets
    y = subset.loc[subset['labels'].isin(genre_subset), 'y'].values
    
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y, test_size=.2)
    
    # Scaling our data to be between 0 and 1
    X_train /= -80
    X_test /= -80
    
    # Reshaping images to be 128 x 660 x 1
    X_train = X_train.reshape(X_train.shape[0], 128, 660, 1)
    X_test = X_test.reshape(X_test.shape[0], 128, 660, 1)
    
    # One hot encoding our labels
    y_train = to_categorical(y_train, len(genre_subset))
    y_test = to_categorical(y_test, len(genre_subset))
    
    return X_train, X_test, y_train, y_test

In [ ]:
# List of all the genres
genre_list = [ 
    'jazz',
    'reggae',
    'rock',
    'blues',
    'hiphop',
    'country',
    'metal',
    'classical',
    'disco',
    'pop'
]

In [ ]:
# List of a subset of the genres
genre_subset = [
    'jazz',
    'reggae',
    'rock',
    'blues',
    'hiphop',
    'country',
    'metal',
    'classical',
    'disco',
    'pop'
]

In [ ]:
# Using our function to get our features and targets
X_train, X_test, y_train, y_test = preprocess_mel_spec_data(mel_specs, genre_subset)

In [ ]:
np.random.seed(23456)

cnn_model = Sequential(name='cnn_1')

cnn_model.add(Conv2D(filters=16,
                     kernel_size=(3,3),
                     activation='relu',
                     input_shape=(128,660,1)))

cnn_model.add(MaxPooling2D(pool_size=(2,4)))

cnn_model.add(Conv2D(filters=32,
                     kernel_size=(3,3),
                     activation='relu'))

cnn_model.add(MaxPooling2D(pool_size=(2,4)))

cnn_model.add(Flatten())

cnn_model.add(Dense(64, activation='relu'))

cnn_model.add(Dropout(0.25))

cnn_model.add(Dense(10, activation='softmax'))

cnn_model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

history = cnn_model.fit(X_train,
                        y_train, 
                        batch_size=16,
                        validation_data=(X_test, y_test),
                        epochs=17)

In [ ]:
# Checking the model summary
cnn_model.summary()

In [ ]:

train_loss = history.history['loss']
test_loss = history.history['val_loss']

plt.figure(figsize=(15, 6))

plt.plot(train_loss, label='Training Loss', color='blue')
plt.plot(test_loss, label='Testing Loss', color='red')

plt.title('Training and Testing Loss by Epoch', fontsize = 25)
plt.xlabel('Epoch', fontsize = 18)
plt.ylabel('Categorical Crossentropy', fontsize = 18)
plt.xticks(range(1,18), range(1,18))

plt.legend(fontsize = 18);

In [ ]:

train_loss = history.history['accuracy']
test_loss = history.history['val_accuracy']


plt.figure(figsize=(15,6))

plt.plot(train_loss, label='Training Accuracy', color='blue')
plt.plot(test_loss, label='Testing Accuracy', color='red')

# Set title
plt.title('Training and Testing Accuracy by Epoch', fontsize = 25)
plt.xlabel('Epoch', fontsize = 18)
plt.ylabel('Accuracy', fontsize = 18)
plt.xticks(range(1,18), range(1,18))

plt.legend(fontsize = 18);

In [ ]:
# Making predictions from the cnn model
predictions = cnn_model.predict(X_test, verbose=1)
# Calculating the confusion matrix 

conf_matrix = confusion_matrix(np.argmax(y_test, 1), np.argmax(predictions, 1))
confusion_df = pd.DataFrame(conf_matrix)

In [ ]:

genre_labels = {
    0:'jazz',
    1:'hiphop',
    2:'country',
    3:'metal',
    4:'classical',
    5:'disco',
    6:'pop'
}
confusion_df = confusion_df.rename(columns=genre_labels)
confusion_df.index = confusion_df.columns
confusion_df